#### Лаба 3

#### задачи траекторного управления

Подробное описание вывода формул в [trajectory_control_LAB3.md](./specifications/trajectory_control_LAB3.md)

In [3]:
import sympy as sm
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from IPython.display import display, Math
import inspect
import bisect

In [25]:
def display_expr(expr):
    frame = inspect.currentframe()
    frame = inspect.getouterframes(frame)[1]
    string = inspect.getframeinfo(frame[0]).code_context[0].strip()
    args = string[string.find('(') + 1:-1].split(',')
    
    names = []
    for i in args:
        if i.find('=') != -1:
            names.append(i.split('=')[1].strip())
        else:
            names.append(i)
    if names[0]=='phi_1':
        names[0] = '\\varphi_1'
    if names[0]=='phi_2':
        names[0] = '\\varphi_2'
    if names[0]=='phi_3':
        names[0] = '\\varphi_3'
    if names[0]=='vm':
        names[0] = 'v_m'
    if names[0]=='vb':
        names[0] = 'v_b'
    if names[0]=='dots':
        names[0] = '\\dot s'
    if names[0]=='dote':
        names[0] = '\\dot e'
    if names[0]=='xi':
        names[0] = '\\xi'
    if names[0]=='bRm':
        names[0] = '{}^{b}R_m'
    if names[0]=='gamma':
        names[0] = '\\gamma'
    if names[0]=='dphidq':
        names[0] = '\\frac{\\partial \\varphi(q)}{\\partial q}'

    expr = expr.xreplace({n : round(n, 3) for n in expr.atoms(sm.Number)})
    str = sm.latex(expr)
    # str = str.replace('\\theta','\\vartheta')
    str = str.replace('vx','v_x')
    str = str.replace('vy','v_y')
    str = str.replace('Fx','F_x')
    str = str.replace('Fy','F_y') 
    str = str.replace('\\alpha_{asterisk}','\\alpha^{*}') 
    str = str.replace('dotxi','\\dot \\xi')
    str = str.replace('\\xi_{dxds}','\\frac{\partial x_{\\xi}(s)}{\partial s}') 

    display(Math(names[0] + '=' + str))    

### Task 1

$m\ddot q = F$

$\dot q=R_O^I(\alpha)v$
\begin{bmatrix} \end{bmatrix}
$R_O^I = \begin{bmatrix} cos\alpha & sin\alpha \cr -sin\alpha & cos\alpha \end{bmatrix}$

Control low:

$F=m*(\dot{\overline{v}} - k_q(\dot q - \overline{v}))$

$\overline{v}=u_e+u_s$

$u_s=R_O^I(\alpha)\gamma^{-1}(q)  \begin{bmatrix} V^* \cr 0 \end{bmatrix}$

$\large u_e = -k_e \varphi(q)\frac{\partial \varphi(q)}{\partial q}$

In [14]:
m = 2.8 # in kg
x, y, alpha = sm.symbols("x y alpha")
x0 = -7
y0 = 4
alpha0 = 3*np.pi/4
vs_des1 = 1 
vs_des2 = 3
vs_des3 = 5

R1 = 3
xc1 = -3
yc1 = 2
R3 = 5
xc3 = 12
yc3 = 2
phi_1 = (x-xc1)**2+(y-yc1)**2-R1**2
phi_2 = -2*sm.cos(0.5*x+0.3)+y-2
phi_3 = (x-xc3)**2 + (y-yc3)**2 - R3**2

bRm = sm.Matrix( [[sm.cos(alpha),sm.sin(alpha)],[-sm.sin(alpha),sm.cos(alpha)]] ) # from mobile to base
display_expr(bRm)
display_expr(phi_1)
display_expr(phi_2)
display_expr(phi_3)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
bRm.subs(alpha,alpha0)

Matrix([
[-0.707106781186547,  0.707106781186548],
[-0.707106781186548, -0.707106781186547]])

In [16]:
s_all = [0, 7.243989865398947, 28.737350135440845, 42.94173297125839, 58.64969623920736]
Tdes1 = s_all[-1]/vs_des1
print(Tdes1) # simulation time
Tdes2 = s_all[-1]/vs_des2
print(Tdes2) # simulation time
Tdes3 = s_all[-1]/vs_des3
print(Tdes3) # simulation time

58.64969623920736
19.54989874640245
11.729939247841472


In [18]:
# times for switching the control
switch_times = []
for i in range(1,4):
    switch_times.append(s_all[i]/vs_des1)
print(switch_times)
switch_times = []
for i in range(1,4):
    switch_times.append(s_all[i]/vs_des2)
print(switch_times)
switch_times = []
for i in range(1,4):
    switch_times.append(s_all[i]/vs_des3)
print(switch_times)

[7.243989865398947, 28.737350135440845, 42.94173297125839]
[2.4146632884663157, 9.579116711813615, 14.313910990419464]
[1.4487979730797895, 5.747470027088169, 8.588346594251679]


In [26]:
gamma = sm.Matrix([[sm.diff(phi_1,y),-sm.diff(phi_1,x)],[sm.diff(phi_1,x),sm.diff(phi_1,y)]])
display_expr(gamma)
dphidq = sm.Matrix([[sm.diff(phi_1,x)],[sm.diff(phi_1,y)]])
display_expr(dphidq)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [28]:
gamma = sm.Matrix([[sm.diff(phi_2,y),-sm.diff(phi_2,x)],[sm.diff(phi_2,x),sm.diff(phi_2,y)]])
display_expr(gamma)
dphidq = sm.Matrix([[sm.diff(phi_2,x)],[sm.diff(phi_2,y)]])
display_expr(dphidq)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [27]:
gamma = sm.Matrix([[sm.diff(phi_3,y),-sm.diff(phi_3,x)],[sm.diff(phi_3,x),sm.diff(phi_3,y)]])
display_expr(gamma)
dphidq = sm.Matrix([[sm.diff(phi_3,x)],[sm.diff(phi_3,y)]])
display_expr(dphidq)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [29]:
kq = 3
ke = 1

### Task 2

In [31]:
x, y,z, alpha = sm.symbols("x y z alpha")
phi_1 = 0.4*x**2 + 0.8*y**2 -225
phi_2 = z+0.25*y**2-3

In [32]:
gradphi_1 = sm.Matrix([[sm.diff(phi_1,x)],[sm.diff(phi_1,y)],[sm.diff(phi_1,z)]])
gradphi_2 = sm.Matrix([[sm.diff(phi_2,x)],[sm.diff(phi_2,y)],[sm.diff(phi_2,z)]])
display_expr(gradphi_1)
display_expr(gradphi_2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>